In [10]:


import numpy as np
import tensorflow as tf

from drowsiness_detection import config
from drowsiness_detection.models import build_dummy_tf_classifier, ThreeDStandardScaler, build_dense_model, build_lstm_model, build_cnn_model


In [2]:
window_size = 10
recording_frequency = 30
exclude_by = "a"

config.set_paths(recording_frequency, window_size)
config.PATHS.WINDOW_DATA

PosixPath('/media/tim/My Passport/drowsiness_data/Windows/Windows_30_Hz/WindowData/10_sec')

In [3]:
num_samples = 20
X = np.random.random(num_samples * 300 * 23).reshape((num_samples, 300, 23))
y = np.concatenate((np.zeros((num_samples // 2)), np.ones((num_samples // 2))))
print(X.shape, y.shape)

(20, 300, 23) (20,)


In [4]:
scaler = ThreeDStandardScaler()
X_scaled = scaler.fit_transform(X, y)
print(X_scaled.shape)

(20, 300, 23)


In [5]:
from collections import namedtuple

Input = namedtuple("Input", "X y")
input = Input(X_scaled, y)

In [6]:
def test_model(model, input: Input = input):
    batch_size = 2
    epochs = 2
    X = tf.convert_to_tensor(input.X, dtype=tf.float64)
    y = tf.convert_to_tensor(input.y, dtype=tf.int8)
    model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    score = model.evaluate(X_scaled, y)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

In [7]:
test_model(model=build_dense_model(input_shape=X.shape))

Epoch 1/2
9/9 [==============================] - 1s 18ms/step - loss: 0.8379 - accuracy: 0.5000 - val_loss: 0.2941 - val_accuracy: 1.0000
Epoch 2/2
1/1 [==============================] - 0s 127ms/step - loss: 0.0281 - accuracy: 1.0000
Test loss: 0.02810894325375557
Test accuracy: 1.0


In [8]:
test_model(model=build_dummy_tf_classifier(input_shape=X.shape))

Activation was set to None which is not supported.
Epoch 1/2
9/9 [==============================] - 0s 15ms/step - loss: 4.8302 - accuracy: 0.3333 - val_loss: 15.4249 - val_accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - 0s 102ms/step - loss: 5.3724 - accuracy: 0.6500
Test loss: 5.37237548828125
Test accuracy: 0.6499999761581421


In [11]:
test_model(model=build_cnn_model(input_shape=X.shape))

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 300, 23)]         0         
                                                                 
 reshape (Reshape)           (None, 300, 23, 1)        0         
                                                                 
 conv2d (Conv2D)             (None, 99, 23, 32)        192       
                                                                 
 batch_normalization (BatchN  (None, 99, 23, 32)       128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 99, 23, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 95, 23, 32)        5152      
                                                           

In [12]:
test_model(model=build_lstm_model(input_shape=X.shape))

Epoch 1/2
9/9 [==============================] - 4s 208ms/step - loss: 0.7000 - accuracy: 0.4444 - val_loss: 0.6142 - val_accuracy: 1.0000
Epoch 2/2
1/1 [==============================] - 1s 654ms/step - loss: 0.1170 - accuracy: 1.0000
Test loss: 0.11697981506586075
Test accuracy: 1.0
